<a href="https://colab.research.google.com/github/MahnazNmz/data-analysis/blob/master/workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build your own agent with Toqan

This notebook provides you with everything you need to know for building your own agent. Through the different sections you will get to know the different aspects of building an agent. In the last section you are challenged to build your own agent with the knowledge you have gained.

## Table of contents
0. Set-up
1. Quick Start
2. Upload files to Toqan
3. Prompt variables
4. Using tools
5. HTTP Request Tool
6. Create your own agent
7. Appendix: Keeping track of conversations & configurations in this notebook


## Environment Setup & Boilerplate Code
You can run the next 5 cells without needing to undersand the code in them. The classes and functions are used to interact with the Toqan API, and create an easy-to-use layer around the api.

In [ ]:
#@title # Download example files used in this notebook
%%capture
!gdown --id 1HDK166C38d8Xa628XR4nmwuTuekJc_i8
!gdown --id 1LaHDapvB5gxWtnGyADyVfHJi6raS9-oy

In [ ]:
#@title # imports & logging
import os
import uuid

import requests
import json
from pydantic import BaseModel
from typing import Optional, List
from datetime import datetime
import logging
import mimetypes
from PIL import Image
import matplotlib.pyplot as plt

# Logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
#@title # Response models of API
class CreateConversationResponse(BaseModel):
    conversation_id: Optional[str] = None
    message: Optional[str] = None

class ContinueConversationResponse(BaseModel):
    message: Optional[str] = None

class ApiAttachmentFile(BaseModel):
    name: str
    mime_type: str

class ApiConversationItemAttachment(BaseModel):
    name: str
    mime_type: str
    image_width: Optional[int] = None
    image_height: Optional[int] = None

class ApiConversationItem(BaseModel):
    id: str
    type: str
    timestamp: datetime
    message: Optional[str] = None
    author_id: Optional[str] = None
    attachments: Optional[List[ApiConversationItemAttachment]] = None

class ApiUploadFileResponse(BaseModel):
    url: Optional[str] = None

class ApiDownloadFileResponse(BaseModel):
    file_name: Optional[str] = None

class Configuration(BaseModel):
    system_prompt: str
    ocr: bool
    url_resolver: bool
    web_search: bool
    text_to_image: bool
    code_executor: bool
    long_text_file_handling: bool
    transcription: bool
    docx_generation: bool
    find_commerce: bool
    http_request: bool

class File(BaseModel):
    file_name: str

class ToolCall(BaseModel):
    name: str
    arguments: str
    result_content: Optional[str]
    result_files: Optional[List[File]]
    completed_at: Optional[datetime]
    error: Optional[str]
    state: str

In [ ]:
#@title API Wrapper
class APIWrapper:
  api_url: str = "https://api.coco.prod.toqan.ai"
  api_key: str
  headers: dict = {
      "X-Api-Key": None,
      "Content-Type": "application/json"
  }

  class Endpoints:
    create_conversation = "/create_conversation"
    create_configuration = "/create_configuration"
    get_configuration = "/get_configuration"
    continue_conversation = "/continue_conversation"
    find_conversation = "/find_conversation"
    get_upload_file_url = "/get_upload_file_url"
    get_download_file_url = "/get_download_file_url"
    get_tool_invocations = "/get_tool_invocations"

  def _attach_file_to_conversation(self, conversation_id: str, file_names: List[str]) -> List[dict]:
      files = []
      for file_name in file_names:
        response = self.upload_file(
            conversation_id,
            file_name)
        if response.url:
            files.append(ApiAttachmentFile(name=file_name, mime_type=get_mime_type(file_name)).dict())
        else:
            logging.error(f"failed to attach file {file_name}")

      return files

  def set_api_key(self, email: str):
    if not email:
      logging.error("email cannot be empty")
      return None

    url = f"{self.api_url}/get_api_key"
    data = {
      "email": email
    }

    response = requests.post(url, headers=self.headers, data=json.dumps(data))
    if response.status_code == 200:
      self.api_key = response.json()["api_key"]
      self.headers["X-Api-Key"] = self.api_key
    else:
      logging.error(f"could not get api key, status: {response.status_code}")
      return None

  def create_configuration(
      self,
      configuration: Configuration,
  ) -> Optional[str]:
      url = f"{self.api_url}{self.Endpoints.create_configuration}"
      data = configuration.dict()
      response = requests.post(url, headers=self.headers, data=json.dumps(data))

      if response.status_code == 200:
          return response.json()["configuration_id"]
      else:
          logging.error(f"could not create configuration, status: {response.status_code}")
          return None

  def get_configuration(self, configuration_id: str) -> Configuration:
      url = f"{self.api_url}{self.Endpoints.get_configuration}"
      data = {
          "configuration_id": configuration_id
      }
      response = requests.post(url, headers=self.headers, data=json.dumps(data))

      if response.status_code == 200:
          return Configuration(**response.json())
      else:
          logging.error(f"could not get configuration, status: {response.status_code}")
          return Configuration()

  def create_conversation(
      self,
      conversation_id: str,
      message: str,
      configuration_id: str = None, # set this if you want to use a configuration you created yourself
      file_names: List[str] = None,
  ):
    url = f"{self.api_url}{self.Endpoints.create_conversation}"
    data = {
      "conversation_id": conversation_id,
      "user_message": message
    }

    if configuration_id:
      data["configuration_id"] = configuration_id

    if file_names:
        data["files"] = self._attach_file_to_conversation(conversation_id, file_names)

    response = requests.post(url, headers=self.headers, data=json.dumps(data))

    if response.status_code == 200:
      logger.info("successfully created conversation")
    else:
      logging.error(f"could not create conversation, status: {response.status_code}, text: {response.text}")

  def continue_conversation(
      self,
      conversation_id: str,
      message: str,
      file_names: List[str] = None,
  ):
    url = f"{self.api_url}{self.Endpoints.continue_conversation}"
    data = {
      "conversation_id": conversation_id,
      "user_message": message
    }

    if file_names:
        data["files"] = self._attach_file_to_conversation(conversation_id, file_names)

    response = requests.post(url, headers=self.headers, data=json.dumps(data))

    if response.status_code == 204:
      logger.info("successfully continued conversation")
    else:
      logging.error(f"could not continue conversation, status: {response.status_code}")

  def find_conversation(self, conversation_id: str) -> List[ApiConversationItem]:
    url = f"{self.api_url}{self.Endpoints.find_conversation}"
    data = {
      "conversation_id": conversation_id
    }

    response = requests.post(url, headers=self.headers, data=json.dumps(data))

    if response.status_code == 200:
      return [ApiConversationItem(**item) for item in response.json()]
    else:
      logging.error(f"could not find conversation, status: {response.status_code}, text: {response.text}")
      return []

  def upload_file(
      self,
      conversation_id: str,
      file_name: str
    ) -> ApiUploadFileResponse:
      url= f"{self.api_url}{self.Endpoints.get_upload_file_url}"
      data = {
        "conversation_id":conversation_id,
        "file_name":file_name
    }
      upload_url_response = requests.post(url, headers=self.headers, data=json.dumps(data))

      if upload_url_response.status_code != 200:
          logging.error(f"failed to generate upload url for file {file_name}, status: {upload_url_response.status_code}")
          return ApiUploadFileResponse()

      upload_url = upload_url_response.json()["url"]
      try:
        with open(file_name, "rb") as object_file:
            object_bytes = object_file.read()
            put_file_response = requests.put(upload_url, data=object_bytes)
            if put_file_response.status_code != 200:
                logging.error(f"failed to put file {file_name}, status: {put_file_response.status_code}")
                return ApiUploadFileResponse()
      except FileNotFoundError:
          logging.error(f"cannot find local file {file_name}")
          return ApiUploadFileResponse()

      return ApiUploadFileResponse(url=upload_url)

  def download_file(
      self,
      conversation_id: str,
      file_name: str,
      local_file_name: str = None
    ) -> ApiDownloadFileResponse:

      url= f"{self.api_url}{self.Endpoints.get_download_file_url}"
      data = {
        "conversation_id":conversation_id,
        "file_name":file_name
    }
      download_url_response = requests.post(url, headers=self.headers, data=json.dumps(data))

      if download_url_response.status_code != 200:
          logging.error(f"failed to get download url for file {file_name}, status: {download_url_response.status_code}")
          return ApiDownloadFileResponse()

      download_url = download_url_response.json()["url"]

      downloaded_content = requests.get(download_url)

      if downloaded_content.status_code != 200:
          logging.error(f"could not download file content for file {file_name}, status: {downloaded_content.status_code}")
          return ApiDownloadFileResponse()
      try:
          if local_file_name:
              file_name = local_file_name
          with open(file_name, "wb") as f:
              f.write(downloaded_content.content)
      except Exception:
          logging.error(f"failed to write the file {file_name} to local {local_file_name}")

      return ApiDownloadFileResponse(file_name=file_name)

  def get_tool_invocations(self, conversation_id: str) -> List[ToolCall]:
    url = f"{self.api_url}{self.Endpoints.get_tool_invocations}"
    data = {
      "conversation_id": conversation_id
    }

    response = requests.post(url, headers=self.headers, data=json.dumps(data))

    if response.status_code == 200:
      return [ToolCall(**item) for item in response.json()]
    else:
      logging.error(f"could not get tool invocations, status: {response.status_code}, text: {response.text}")
      return []

# class needed for encoding things
class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)

def get_mime_type(file_name: str) -> str:
    t = mimetypes.guess_type(file_name)[0] or "application/octet-stream"
    return t

In [ ]:
#@title Variables and functions used in the notebook
# Set up some variables to this notebook to keep track of conversations
if "all_conversations_created" not in globals():
  all_conversations_created = [] # used to keep track of all conversation ids, in case you want to review later

if "latest_conversation_id" not in globals():
  latest_conversation_id = None

if "all_configurations_created" not in globals():
  all_configurations_created = []

if "latest_configuration_id" not in globals():
  latest_configuration_id = None

def create_conversation_id():
  global latest_conversation_id
  global all_conversations_created

  latest_conversation_id = str(uuid.uuid4())
  all_conversations_created.append(latest_conversation_id)

def create_configuration(configuration: Configuration):
  global latest_configuration_id
  global all_configurations_created

  configuration_id = api.create_configuration(configuration=configuration)

  if configuration_id:
    all_configurations_created.append(configuration_id)
    latest_configuration_id = configuration_id

def preview_image(image_path):
      try:
          image = Image.open(image_path)
          plt.figure()
          plt.imshow(image)
          plt.axis('off')
          plt.show()  # Display the image using matplotlib
      except Exception as e:
          logger.info(f"File {image_path} cannot be previewed, view it in your file system attached to this notebook.")

def preview_text_files(text_file_paths):
    for file_path in text_file_paths:
        try:
            with open(file_path, 'r') as file:
                lines = file.readlines()[:5]
                preview = ''.join(lines)[:100]
                print(f'Preview of {file_path}:\n{preview}\n--------------\n')
        except Exception as e:
            logger.info(f"File {file_path} cannot be previewed, view it in your file system attached to this notebook.")

def preview_other_files(file_paths):
    for file_path in file_paths:
        logger.info(f"File {file_path} cannot be previewed, view it in your file system attached to this notebook.")

def preview_files(file_paths):
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff')
    text_extensions = ('.txt', '.md', '.py', '.csv')

    image_paths = []
    text_paths = []
    other_paths = []

    for file_path in file_paths:
        mime_type, _ = mimetypes.guess_type(file_path)
        if mime_type:
            if mime_type.startswith('image'):
                image_paths.append(file_path)
            elif mime_type.startswith('text') or file_path.endswith(text_extensions):
                text_paths.append(file_path)
            else:
                other_paths.append(file_path)
        else:
            if file_path.endswith(image_extensions):
                image_paths.append(file_path)
            elif file_path.endswith(text_extensions):
                text_paths.append(file_path)
            else:
                other_paths.append(file_path)

    if image_paths:
        preview_image(image_paths[0])
        if len(image_paths) > 1:
            logger.info(f"Only dipslaying first image files, view others in file system attached to this notebook: {image_paths[1:]}")
    if text_paths:
        preview_text_files(text_paths)
    if other_paths:
        preview_other_files(other_paths)


def download_files_from_response(conversation_item: ApiConversationItem) -> List[str]:
  global latest_conversation_id

  local_folder = "downloaded_files"
  if not os.path.exists(local_folder):
      os.makedirs(local_folder)

  if conversation_item.author_id != "Toqan":
    # logger.info("Last message was not from Toqan, no files to download")
    return []

  all_local_file_paths = []
  if conversation_item.attachments:
    logger.info("Downloading attachments")
    for f in conversation_item.attachments:
        toqan_file_name = f.name
        local_file_path = os.path.join(local_folder, toqan_file_name)
        if not os.path.exists(local_file_path):
            api.download_file(latest_conversation_id, toqan_file_name, local_file_path)
        else:
            logger.info(f"File already exists: {toqan_file_name}, not downloading again")

        api.download_file(latest_conversation_id, toqan_file_name, local_file_path)
        all_local_file_paths.append(local_file_path)
  else:
    logger.info("No attachments found in the last message")

  return all_local_file_paths

## 0. Set-up

Fill in your email in the email variable in the next cell, and run the cell to set up your connection with the Toqan API

In [ ]:
email = "" # your email here
api = APIWrapper()
api.set_api_key(email)

## 1. Quick Start

The following cells will show you how to create an agent by creating a configuration for Toqan, start a conversation, and see the answer. In the detailed section more sophisticated use of this notebook is explained.

An explanation on why `{tool_guidelines}` is added to the system prompt is in section 3 of this notebook.

In [ ]:
# Create a new configuration
create_configuration(Configuration(
    system_prompt="You are a clown, always announce you are a clown and then respond in a clown way. However, do not use emojis. {tool_guidelines}",
    ocr=False,
    url_resolver=False,
    web_search=False,
    text_to_image=False,
    code_executor=False,
    long_text_file_handling=True, # note that this tool allows Toqan to process text files you upload
    transcription=False,
    docx_generation=False,
    find_commerce=False,
    http_request=False,
))
print(f"configuration_id: {latest_configuration_id}")

In [ ]:
# Create a new conversation_id, this is a prerequisite for creating a new conversation
create_conversation_id()
print(f"latest_conversation_id: {latest_conversation_id}\n")

# Create a new conversation with Toqan
response = api.create_conversation(
  conversation_id=latest_conversation_id, # auto-set to the last created
  message="how are you now?",
  configuration_id=latest_configuration_id, # auto-set to the last created
)

In [ ]:
# Check the conversation, if an answer is ready, it will appear here
conversation_messages = api.find_conversation(latest_conversation_id)
print(json.dumps([item.dict() for item in conversation_messages], cls=DateTimeEncoder, indent=2))

# any attachments in the last message will be downloaded to local folder "downloaded_files"
last_message = conversation_messages[-1]
local_file_paths = download_files_from_response(last_message)

# NOTE THAT responses from Toqan might not be immediately available. Re-run this cell in a few seconds in that case.

## 2. Upload files to Toqan
Toqan is able to use files. Most file types are supported in some way, because of the tools that are available. If you have enabled the file_text_handling tool, Toqan can i.e. use a tool to handle long text files. The code executor tool is able to load files and execute code on them.

Toqan is also able to "see" images, so if you upload an image, Toqan knows about the contents automatically and you can ask it questions about it.

In order to upload files to toqan, you need to put the file you want to upload into this folder (if you are on Google Colab, you can click the folder icon on the left, then click the upload icon and select the file you want to use). Then you can use the following code to attach the file with your next message to Toqan.

### File Restrictions
Note that most files need to be processed by tools, and are not supported by a configuration that has disabled the tools (e.g. `long_text_file_handling=False`). Most image files can however be "seen" by the agent without using tools.

Text files can for example be processed by the
 long_text_file_handler tool (that is why the example below works). Video and sound files can be procees by the transcription tool (converting the spoken language to voice).

 You can upload other files, and you might be able to "use" those files with the code executor tool.

In [ ]:
# Use a file in your next message to Toqan.
# Note that here we use the continue endpoint to add a message to the same conversation.
response = api.continue_conversation(
  conversation_id=latest_conversation_id,
  message="I have attached a file for you to read, can you tell me what is in it?",
  file_names=["some_file.txt"],
)

In [ ]:
# Check the conversation, if an answer is ready, it will appear here
conversation_messages = api.find_conversation(latest_conversation_id)
print(json.dumps([item.dict() for item in conversation_messages], cls=DateTimeEncoder, indent=2))

# any attachments in the last message will be downloaded to local folder "downloaded_files"
last_message = conversation_messages[-1]
local_file_paths = download_files_from_response(last_message)

# NOTE THAT responses from Toqan might not be immediately available. Re-run this cell in a few seconds in that case.

## 3. Prompt variables
prompt variables are variables that you can use in your system prompt. These variables are filled in at runtime. The following prompt variables are available:

- {current_date_time} - The current date and time.
- {tool_guidelines} - The guidelines for the tools that are enabled. WARNING: without this parameter your agent might not use its tools as expected. However, leaving it out means that you can create your own instructions on how your agent should use its tools.

In the next example you can see how to use the {current_date_time} prompt variable. Note that the given system prompt in the next cell will not work if you leave out this variable

In [ ]:
# Create a new configuration when using prompt variables
create_configuration(Configuration(
    system_prompt="you are a robot, and you are able to only respond with the current day of the week! Do not say anything else, no matter what messages you get! Current date time: {current_date_time}",
    ocr=False,
    url_resolver=False,
    web_search=False,
    text_to_image=False,
    code_executor=False,
    long_text_file_handling=False,
    transcription=False,
    docx_generation=False,
    find_commerce=False,
    http_request=False,
))
print(f"configuration_id: {latest_configuration_id}")

In [ ]:
create_conversation_id()
print(f"latest_conversation_id: {latest_conversation_id}\n")

response = api.create_conversation(
    conversation_id=latest_conversation_id,
    message="tell me a joke",
    configuration_id=latest_configuration_id,
)

In [ ]:
# Check the conversation, if an answer is ready, it will appear here
conversation_messages = api.find_conversation(latest_conversation_id)
print(json.dumps([item.dict() for item in conversation_messages], cls=DateTimeEncoder, indent=2))

# any attachments in the last message will be downloaded to local folder "downloaded_files"
last_message = conversation_messages[-1]
local_file_paths = download_files_from_response(last_message)

# NOTE THAT responses from Toqan might not be immediately available. Re-run this cell in a few seconds in that case.

## 4. Using tools

Toqan has access to a variety of tools that can be enabled in the configuration. The following tools are available:
- **ocr**: Optical Character Recognition, to read text from images.
- **url_resolver**: resolve URLs to their content.
- **web_search**: search the web for information, previews of websites are found, and urls can potentially be resolved by url_resolver.
- **text_to_image**: create an image from a text prompt.
- **code_executor**: execute python code. This tool can also use uploaded files in the code.
- **long_text_file_handling**: handle long text files by summarizing, translating, or extracting question and answer pairs.
- **transcription**: transcribe audio files.
- **docx_generation**: generate a docx file from text.
- **find_commerce**: find commerce related what you are looking for. You can be specific and i.e. ask for a store in Amsterdam that sells shoes, and opens before 10am on a Saturday.
- **http_request**: make http requests to any specified endpoint. NOTE that this is a special tool for which you need to do the configuration in the system prompt. More info at the http request tool section of this notebook.

If you are enabling tools in your configuration, use the prompt variable **{tool_guidelines}** in your system prompt. This will insert the detailed guidelines for the tools that are enabled. If you do not use this prompt variable, your agent might not use its tools as expected. However, leaving it out means that you can create your own instructions in the system prompt on how your agent should use its tools.

### See tool call invocations in a conversation
If you want to see what your agent is doing under the hood, you can inspect which tool invocations it made. This can be done with the get_tool_invocations function of the api wrapper. See the fourth cell in this section for an example.

### Image variation tool
In the next example, we will create an image variation tool. We use the standard capability of Toqan to "view" images we submit, plus the tool to generate images from a prompt.


In [ ]:
configuration = create_configuration(Configuration(
    system_prompt="You are an image conversion agent. When someone shows you an image, create a new image with the text to image tool that has the same subject, but a blue color scheme. Return that image to the user. You should not respond to any other requests from users. If no image is uploaded, ask the user to upload an image and explain that your only capability is image variation. {tool_guidelines}",
    ocr=False,
    url_resolver=False,
    web_search=False,
    text_to_image=True, # enable text to image tool
    code_executor=False,
    long_text_file_handling=False,
    transcription=False,
    docx_generation=False,
    find_commerce=False,
    http_request=False,
))

In [ ]:
create_conversation_id()
print(f"latest_conversation_id: {latest_conversation_id}\n")
response = api.create_conversation(
    conversation_id=latest_conversation_id,
    message="I have attached an image for you to convert",
    configuration_id=latest_configuration_id,
    file_names=["cat.png"]
)

In [ ]:
# Check the conversation, if an answer is ready, it will appear here
conversation_messages = api.find_conversation(latest_conversation_id)
print(json.dumps([item.dict() for item in conversation_messages], cls=DateTimeEncoder, indent=2))

# any attachments in the last message will be downloaded to local folder "downloaded_files"
last_message = conversation_messages[-1]
local_file_paths = download_files_from_response(last_message)

# NOTE THAT responses from Toqan might not be immediately available. Re-run this cell in a few seconds in that case.

In [ ]:
preview_files(local_file_paths)

In [ ]:
# Check which tool calls are made in this conversation by the agent
tools = api.get_tool_invocations(latest_conversation_id)
print(json.dumps([tool.dict() for tool in tools], cls=DateTimeEncoder, indent=2))

## 5. HTTP Request Tool

The HTTP Request tool allows you to make HTTP requests to any specified endpoint. This is a special tool that requires configuration in the system prompt. The system prompt should describe the following information:
  - **url**: The URL to which the request can be made.
  - **method**: The HTTP method to use for the request (e.g., GET, POST, PUT, DELETE).
  - **authorization**: The authorization header to use for the request (e.g., Bearer token: abcdefg12395).
  - **content_type**: The content type of the request (e.g., application/json).
  - **body**: The body of the request.
  
The latter three parameters are optional. This specific HTTP Request tool does not need to have the **{tool_guidelines}** prompt variable in the system prompt, since we are creating the guidelines on how to use this tool ourselves!

Let's consider a case where we want to query the latest information about star wars characters by using an api. We use the Api instead of the general LLM knowledge because more details are available in the API. We can use the swapi.dev api to get information about star wars characters. The system prompt can be formatted as follows:
```
Whenever the user asks something about Star Wars, lookup the characters using the http_request tool calling swapi.
Use the response to answer the question.

Method: GET
URL: https://swapi.dev/api/people
Authorization: Bearer token: abcdefg12395
Content-Type: application/json
Body: {}
```

The example above uses a url that actually does not need to specify authorization, content-Type and body.
Look at the following example on how to actually use it.


In [ ]:
create_configuration(Configuration(
    system_prompt="Whenever the user asks something about Star Wars, lookup the characters using the http_request tool calling swapi. Use the response to answer the question. Method: GET URL: https://swapi.dev/api/people",
    ocr=False,
    url_resolver=False,
    web_search=False,
    text_to_image=False,
    code_executor=False,
    long_text_file_handling=False,
    transcription=False,
    docx_generation=False,
    find_commerce=False,
    http_request=True,

))

In [ ]:
create_conversation_id()
print(f"latest_conversation_id: {latest_conversation_id}\n")
response = api.create_conversation(
    conversation_id=latest_conversation_id,
    message="Tell me about Luke Skywalker",
    configuration_id=latest_configuration_id,
)

In [ ]:
# Check the conversation, if an answer is ready, it will appear here
conversation_messages = api.find_conversation(latest_conversation_id)
print(json.dumps([item.dict() for item in conversation_messages], cls=DateTimeEncoder, indent=2))

# NOTE THAT responses from Toqan might not be immediately available. Re-run this cell in a few seconds in that case.

In [ ]:
# Check which tool calls are made in this conversation by the agent
tools = api.get_tool_invocations(latest_conversation_id)
print(json.dumps([tool.dict() for tool in tools], cls=DateTimeEncoder, indent=2))

## 6. Create your own agent
Now that you know of the possibilities of the notebook, its time to start working on your own ideas. Use the system prompt, existing tools and custom tools via the http request tool to create your own agent.

### Quick overview of the steps
1. Create a configuration with `create_configuration`: set system prompt and enable tools. Don't forget prompt variables and be sure to specify your custom apis if you are using the http request tool.
2. Create a conversation with `create_conversation`: create a conversation id and start a conversation with Toqan. Be sure to include file paths in the files parameter if you want to upload files.
3. Continue the conversation with `continue_conversation`: continue the conversation with Toqan. Again don't forget your files
4. Check the conversation with `find_conversation`: check the conversation to see the messages and download any attachments.

In [ ]:
# 1. Create the configuration
create_configuration(Configuration(
    system_prompt="...",
    ocr=...,
    url_resolver=...,
    web_search=...,
    text_to_image=...,
    code_executor=...,
    long_text_file_handling=...,
    transcription=...,
    docx_generation=...,
    find_commerce=...,
    http_request=...,
))

In [ ]:
# 2. Create the conversation
create_conversation_id()
print(f"latest_conversation_id: {latest_conversation_id}\n")
response = api.create_conversation(
    conversation_id=latest_conversation_id,
    message=...,
    configuration_id=latest_configuration_id,
    file_names=[...]
)

In [ ]:
# 3. Continue the conversation
response = api.continue_conversation(
  conversation_id=latest_conversation_id,
  message=...,
  file_names=[...]
)

In [ ]:
# 4. Check the conversation, if an answer is ready, it will appear here
conversation_messages = api.find_conversation(latest_conversation_id)
print(json.dumps([item.dict() for item in conversation_messages], cls=DateTimeEncoder, indent=2))

In [ ]:
# Check which tool calls are made in this conversation by the agent
tools = api.get_tool_invocations(latest_conversation_id)
print(json.dumps([tool.dict() for tool in tools], cls=DateTimeEncoder, indent=2))

## Appendix: Keeping track of conversations & configurations
This notebook automatically keeps track of all conversations and configurations created. Uncomment the lines to check all conversation or configuration ids that you have created. With the find_conversation and get_configuration functions of the api wrapper you can retrieve the data.

In [ ]:
# Show all ids of conversations created
print(f"all_conversations_created: {all_conversations_created}")

# Check an earlier created conversation (fill in the conversation_id)
conversation = api.find_conversation("YOUR_CONVERSATION_ID_HERE")
print(conversation)

In [ ]:
# Show all ids of configurations created
print(f"all_configurations_created: {all_configurations_created}")

# Check an earlier created configuration (fill in the configuration_id)
configuration = api.get_configuration(latest_configuration_id)
print(configuration)

In [ ]:
# Continue a conversation with Toqan
response = api.continue_conversation(
  conversation_id=latest_conversation_id,
  message="Tell me a joke?"
)